In [1]:
import numpy as np
import pandas as pd
import csv
import random
import math

# Инициализация

In [2]:
drop_column = ['№п/п', 'Дата рождения', 'Учебное заведение', 'Населенный пункт по прописке', 'Основание поступления']
gender_dict = {'Мужской': 1,'Женский': 0}

# количество признаков входного слоя
feature_count = 5

# Считывание данных из файла и их подготовка

In [3]:
# подготовленный датасет
train_data = []
# датасет для тестирования
test_data = []

df = pd.read_csv('data/input.csv', sep=',')
df.drop(drop_column, axis = 1, inplace = True)

# переводим пол в числовой формат, т. к. это будет один из признаков
df['Пол'].replace(gender_dict, inplace = True) 

# приводим баллы по ЕГЭ к диапозону от 0 до 1
df.astype({'Баллы ЕГЭ': 'float'}).dtypes
df['Баллы ЕГЭ'] /= 100

fios = df['ФИО'].unique()
subjects = df['Предметы ЕГЭ'].unique()
classes = df['Специальность/направление'].unique()

for el in fios:
    student = df[df['ФИО'] == el]
    features = np.zeros((6))
    for i in range(len(subjects)):
        res = student.loc[student['Предметы ЕГЭ'] == subjects[i], 'Баллы ЕГЭ'].values
        features[i] = res[0] if len(res) else 0.01
    features[5] = student.iloc[0]['Пол']
    
    # Нормализация входного вектора
    norm_features = np.zeros((6))
    for i in range(len(features)):
        sum = 0 
        for j in range(len(features)):
            sum += features[j] ** 2
        norm_features[i] = features[i] / math.sqrt(sum)
    
    #
    temp_data = dict(fio = el, features = norm_features, 
                    result = np.where(classes == student.iloc[0]['Специальность/направление'])[0].item(0))
    
    # 90% - train, 10% - test
    if (len(train_data) / len(fios)) < 0.9:
        train_data.append(temp_data)
    else:
        test_data.append(temp_data)


0
1
2
3
4
4
5
6
7
0
4
8
6
0
8
6
3
9
7
10
9
8
3
11
12
1
0
0
0
0
13
4
1
0
11
2
3
3
8
8
4
6
4
0
9
8
9
11
0
8
4
0
6
8
2
2
0
1
5
14
6
10
4
2
6
15
8
0
0
7
2
1
2
7
4
3
15
6
12
5
5
3
0
0
1
4
0
11
4
8
2
13
7
1
0
0
16
7
3
1
4
12
10
1
8
6
0
3
3
3
15
3
9
5
17
4
0
3
0
1
2
6
12
4
3
15
4
10
0
4
1
0
5
5
6
11
4
0
5
8
11
2
0
4
9
7
9
8
4
4
10
9
10
9
6
5
9
8
2
8
0
8
3
2
6
8
4
9
1
4
8
15
5
2
8
13
16
3
6
6
4
3
0
10
6
8
9
9
8
0
0
0
8
13
4
6
8
18
4
0
11
3
8
0
1
6
0
7
8
8
0
6
1
7
3
8
8
2
2
2
0
4
7
6
8
15
8
0
10
4
0
1
9
10
0
7
0
4
7
1
0
8
2
8
2
8
1
4
0
0
4
9
4
4
0
4
9
8
0
2
5
0
8
1
8
0
2
0
5
1
0
0
8
11
12
5
7
1
3
9
2
1
11
4
10
5
17
12
9
8
8
0
3
4
12
0
4
0
4
4
12
16
10
6
6
8
16
1
6
9
4
3
3
0
3
9
2
0
0
9
0
3
11
10
10
10
3
6
0
11
1
9
5
0
10
3
0
10
13
7
13
4
2
8
8
4
11
1
1
0
7
0
2
0
4
6
13
8
19
11
4
0
20
13
12
0
0
3
2
2
3
10
6
3
0
6
0
2
8
0
8
6
8
4
8
6
9
0
14
6
12
3
0
3
4
6
0
10
8
0
0
5
3
6
11
8
5
6
8
13
4
2
8
9
10
0
0
5
16
0
14
5
9
13
15
0
2
6
9
12
16
12
3
10
2
4
1
0
3
6
5
12
4


# Обучение

## Кохонен

In [4]:
def getKohonenWinnerIndex(x, neuron_counts, w, s):
    result = 0
    max_net = -1
    for i in range(neuron_counts):
        net = 0
        for j in range(len(x)):
            net += x[j] * w[j][i] * s[i]
        if (net > max_net):
            result = i
            max_net = net
    print(max_net)
    return result

In [5]:
# скорость обучения слоя Кохонена
theta = 0.7
# скорость обучения слоя Гроссберга
beta = 0.1
#количество нейронов выходного слоя (слоя Гроссберга) = количество проффессий 
grossberg_neuron_counts = len(classes)
# количество нейронов кохонена
kohonen_neuron_count = len(classes)
# штрафы кластеров
s_w = np.ones((kohonen_neuron_count))

#инициализация начальных весов слоя Кохонена kohonen_w[x][y]
# x - индекс нейрона входного слоя
# y - индекс нейрона слоя Кохонена
kohonen_w = np.zeros((len(subjects) + 1, kohonen_neuron_count))
for i in range(len(subjects) + 1):
    for j in range(kohonen_neuron_count):
        kohonen_w[i, j] = random.uniform(0.01, 0.99)

# Инициализация начальных весов слоя Гроссберга grossberg_w[x][y]
# x - индекс нейрона слоя Кохонена
# y - индекс нейрона слоя Гроссберга
grossberg_w = np.zeros((kohonen_neuron_count, grossberg_neuron_counts))
for i in range(kohonen_neuron_count):
    for j in range(grossberg_neuron_counts):
        grossberg_w[i, j] = random.uniform(0.01, 0.99)

# Обучение на подготовленном датасете
for student in train_data:
    
    # Определение нейрона победителя
    winner_index = getKohonenWinnerIndex(student['features'], kohonen_neuron_count, kohonen_w, s_w)
    # Увеличиваем штраф, уменьшая его коэффициент
    s_w[winner_index] *= 0.9
    
    # Корректировка весов нейрона победителя на слое Кохонена
    for i in range(feature_count):
        kohonen_w[i][winner_index] += theta * (student['features'][i] - kohonen_w[i][winner_index])
    
    # Корректировка весов для нейронов слоя Гроссберга, соединенных с нейроном победителем слоя Кохонена
    for i in range(grossberg_neuron_counts):
        if i == student['result']:
            grossberg_w[winner_index][i] += beta * (1 - grossberg_w[winner_index][i])
        else:
            grossberg_w[winner_index][i] += beta * (0 - grossberg_w[winner_index][i])
    
    # Изменение скорости обучения
    theta, beta = theta * 0.1, beta * 0.1  

1.5107052473696505
0.9982699987238125
1.4115681431711293
1.3443014574890237
1.1943046008065132
1.3697375426914338
1.3610004917939529
1.225704208176836
1.234075299190748
1.3330505515399995
1.2646244382006522
1.2563586934536222
1.1074891826493427
1.2206826696050006
1.1381310616013562
1.1403862340299828
1.122852382878785
1.0783430067309907
1.1867013634362484
1.0895222841818315
1.045139745664474
1.0020020613810483
1.090181439737724
0.998350956799392
0.9692825197294562
1.0045681140948632
1.0942501290419169
1.0729088901452157
0.8953878032292856
1.0364958642354036
0.9717774785005735
1.024992652156327
1.0811912531483205
0.9872113012796858
0.9379426781174487
0.9083708554571441
0.9769911838760702
0.9890047263123548
0.8694116589005849
0.8762931785117423
0.930539525712287
0.8055744250844443
1.019825315551683
0.8808731607867463
0.8371217116243201
0.8766084990803524
0.9215551545376448
0.8463416668260715
0.9010385015440887
1.0407895157057192
0.8730928823975951
0.8215504071558353
0.8201331497914681
0.

# Тестирование

In [6]:
accuracy = 0
s_w = np.ones((kohonen_neuron_count))

for student in test_data:
    print('ФИО: ' + student['fio'])
    print('Поступил на специальность: ' + classes[student['result']])
    print('Результаты нейросети:')
    # Определение нейрона победителя
    winner_index = getKohonenWinnerIndex(student['features'], kohonen_neuron_count, kohonen_w, s_w)
    best_results = [0.0, 0.0, 0.0]
    best_indexes = [0, 1, 2]
    max_res = 0.0
    max_index = 0
    for i in range(grossberg_neuron_counts):
        
        if (grossberg_w[winner_index][i] > best_results[0]):
            best_results[2] = best_results[1]
            best_results[1] = best_results[0]
            best_results[0] = grossberg_w[winner_index][i]
            best_indexes[2] = best_indexes[1]
            best_indexes[1] = best_indexes[0]
            best_indexes[0] = i
        elif (grossberg_w[winner_index][i] > best_results[1]):
            best_results[2] = best_results[1]
            best_results[1] = grossberg_w[winner_index][i]
            best_indexes[2] = best_indexes[1]
            best_indexes[1] = i
        elif (grossberg_w[winner_index][i] > best_results[2]):
            best_results[2] = grossberg_w[winner_index][i]
            best_indexes[2] = i
        
    print(classes[best_indexes[0]] + ': ' + str(grossberg_w[winner_index][best_indexes[0]] * 100) + '%')
    print(classes[best_indexes[1]] + ': ' + str(grossberg_w[winner_index][best_indexes[1]] * 100) + '%')
    print(classes[best_indexes[2]] + ': ' + str(grossberg_w[winner_index][best_indexes[2]] * 100) + '%')
    
    
    if (student['result'] in best_indexes):
        accuracy += 1
    print('--------------------------------------------------')
print ('Точность: ' + str((accuracy / len(test_data)) * 100) + '%')

ФИО: Хисамиев Харис Мурадович
Поступил на специальность: Информатика и вычислительная техника
Результаты нейросети:
1.3914877862467159
Электроэнергетика и электротехника: 95.28905749169955%
Прикладная информатика (в экономике): 93.43391518301398%
Машиностроение: 92.950263459654%
--------------------------------------------------
ФИО: Хлесткова Мария Александровна
Поступил на специальность: Информатика и вычислительная техника
Результаты нейросети:
1.3281990986105312
Электроэнергетика и электротехника: 95.28905749169955%
Прикладная информатика (в экономике): 93.43391518301398%
Машиностроение: 92.950263459654%
--------------------------------------------------
ФИО: Хомич Игорь Александрович
Поступил на специальность: Конструирование и технология электронных средств
Результаты нейросети:
1.3362156103746075
Сервис: 94.26209899345572%
Информатика и вычислительная техника: 89.77124773699438%
Управление качеством: 81.46478534301775%
--------------------------------------------------
ФИО: Хром